# CYTools Wasm demo

This notebook runs entirely in your browser without the need to install anything and or to have a server somewhere running the computations. The functionality is limited, but it is already pretty decent. More information can be found [here](https://github.com/ariostas/cytools-wasm).

In [ ]:
# This is needed for now, but later on it might be possible to package it better
%pip install cytools python_flint_wasm numpy scipy tqdm matplotlib

In [ ]:
from cytools import read_polytopes
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Here we run the example listed in the tutorial.
# One notable change is that fetch_polytopes is not used because
# currently it is difficult to implement this functionality.
# Also, we currently don't have a way to find the tip of the 
# stretched Kahler cone because none of the backends are available in wasm,
# although it can be done with some work.

h11_list = []
nonzerointnums = []
costhetamin = []
limit_per_h11 = 5
curr_h11 = 0
curr_count = 0

for p in read_polytopes("./sample_polytopes.txt", lattice="N", favorable=True):
    h11 = p.h11(lattice="N")
    if h11 > curr_h11:
        print(f"Processing h11={h11}", end="\r")
        curr_h11 = h11
        curr_count = 0
    curr_count += 1
    if curr_count > limit_per_h11:
        continue
    t = p.triangulate()
    cy = t.get_cy()
    h11_list.append(h11)
    nonzerointnums.append(len(cy.intersection_numbers(in_basis=True)))
    mori_rays = cy.toric_mori_cone(in_basis=True).rays()
    mori_rays_norms = np.linalg.norm(mori_rays, axis=1)
    n_mori_rays = len(mori_rays)
    costhetamin.append(min(
        mori_rays[i].dot(mori_rays[j])
            /(mori_rays_norms[i]*mori_rays_norms[j])
        for i in range(n_mori_rays) for j in range(i+1,n_mori_rays)))
print("Finished processing all h11s!")
print(f"Scanned through {len(h11_list)} CY hypersurfaces.")

In [ ]:
xdata = [h11_list]*2
ydata = [nonzerointnums, costhetamin]
xlabels = [r"$h^{1,1}$"]*2
ylabels = [r"# nonzero $\kappa_{ijk}$", r"$\cos(\theta_{min})$"]
fig, ax0 = plt.subplots(2, 2, figsize=(15,5))

for i,d in enumerate(ydata):
    ax = plt.subplot(121+i)
    ax.scatter(xdata[i], ydata[i], s=10)
    plt.xlabel(xlabels[i], size=20)
    plt.ylabel(ylabels[i], size=20)
    plt.tick_params(labelsize=15, width=2, length=5)

plt.subplots_adjust(wspace=0.3, hspace=0.22)